# EventinatR usage

The following shows a simplified usage of the EventinatR objects.

In [ ]:
#!pwsh
dotnet pack ..\..\EventinatR.sln --output C:\Windows\Temp\EventinatR

Microsoft (R) Build Engine version 17.0.0-preview-21460-01+8f208e609 for .NET
Copyright (C) Microsoft Corporation. All rights reserved.

  Determining projects to restore...
  All projects are up-to-date for restore.


In [ ]:
#i "nuget:C:\Windows\Temp\EventinatR"
#r "nuget:EventinatR"
#r "nuget:System.Memory.Data"
#r "nuget:System.Linq.Async"

using System;
using System.Collections.Generic;
using System.Linq;
using EventinatR;
using EventinatR.InMemory;
using EventinatR.Serialization;

Restore sources C:\Windows\Temp\EventinatR Installed Packages EventinatR, 1.0.0 System.Linq.Async, 5.0.0 System.Memory.Data, 1.0.2

# Domain model

We are modeling a simple group with members, with the ability to add and remove members and the events that change the state of the group.

In [ ]:
public record GroupMember(string Name);
public record GroupId(string Name);

public record GroupEvent
{
    public record Created(GroupId Id) : GroupEvent;
    public record AddedMember(GroupId Id, GroupMember Member) : GroupEvent;
    public record RemovedMember(GroupId Id, GroupMember Member) : GroupEvent;

    public static readonly EventDeserializer Deserializer = new EventDeserializer(builder => 
    {
        builder.UseDefault<Created>();
        builder.UseDefault<AddedMember>();
        builder.UseDefault<RemovedMember>();
    });
}

# Aggregate root

The aggregate root is the group, this is a simple example of what an aggregate root with event sourcing could look like, it is not the best example.

In [ ]:
public class Group
{
    public GroupId Id { get; private set; }
    public IEnumerable<GroupMember> Members => _members.AsEnumerable();
    
    private readonly List<GroupMember> _members = new();
    private readonly List<GroupEvent> _uncommittedEvents = new();

    private Group()
    {
    }

    private void AddEvent(GroupEvent e)
    {
        ApplyEvent(e);
        _uncommittedEvents.Add(e);
    }

    private void ApplyEvent(GroupEvent e)
    {
        switch (e)
        {
            case GroupEvent.Created created:
                Apply(created);
                break;
            case GroupEvent.AddedMember addedMember:
                Apply(addedMember);
                break;
            case GroupEvent.RemovedMember removedMember:
                Apply(removedMember);
                break;
             default:
                 throw new InvalidOperationException($"Unsupported event: {e.GetType().FullName}");
        }
    }

    public static Group Create(string name)
    {
        var group = new Group();
        group.AddEvent(new GroupEvent.Created(new GroupId(name)));
        return group;
    }

    private void Apply(GroupEvent.Created e)
        => Id = e.Id;

    public void AddMember(string name)
    {
        if (!_members.Any(x => x.Name == name))
        {
            var member = new GroupMember(name);
            var e = new GroupEvent.AddedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.AddedMember e)
        => _members.Add(e.Member);

    public void RemoveMember(string name)
    {
        var member = _members.FirstOrDefault(x => x.Name == name);

        if (member is not null)
        {
            var e = new GroupEvent.RemovedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.RemovedMember e)
        => _members.Remove(e.Member);

    public static async Task<Group?> ReadAsync(EventStream stream)
    {
        var events = await stream.ReadAsync().ToListAsync();

        if (!events.Any())
        {
            return null;
        }

        var group = new Group();

        foreach(var e in events)
        {
            if (!GroupEvent.Deserializer.TryDeserialize<GroupEvent>(e, out var groupEvent))
            {
                throw new InvalidOperationException($"The event {e.Type} is not supported.");
            }

            group.ApplyEvent(groupEvent);
        }

        return group;
    }

    public async Task SaveAsync(EventStream stream)
    {
        if (_uncommittedEvents.Count > 0)
        {
            await stream.AppendAsync(_uncommittedEvents);
            _uncommittedEvents.Clear();
        }
    }
}

# Saving events to the stream

1. Initialize an in-memory event store
2. Create the aggregate root and perform some commands to generate events
3. Get a stream for the group from the event store
4. Save the events to the stream

In [ ]:
EventStore store = new InMemoryEventStore();

var group = Group.Create("Final Fantasty Games");

group.AddMember("Final Fantasy I");
group.AddMember("Final Fantasy II");
group.AddMember("Final Fantasy III");
group.AddMember("Final Fantasy IV");
group.AddMember("Final Fantasy V");
group.AddMember("Final Fantasy VI");
group.AddMember("Final Fantasy VII");
group.AddMember("Final Fantasy VIII");
group.AddMember("Final Fantasy IX");
group.AddMember("Final Fantasy X");
group.AddMember("Final Fantasy XI");
group.AddMember("Final Fantasy XII");
group.AddMember("Final Fantasy XIII");
group.AddMember("Final Fantasy XIV");
group.AddMember("Final Fantasy XV");

var stream = await store.GetStreamAsync(group.Id.Name);

await group.SaveAsync(stream);

display(group);

Id,Members
{ GroupId { Name = Final Fantasty Games }: Name: Final Fantasty Games },"[ { GroupMember { Name = Final Fantasy I }: Name: Final Fantasy I }, { GroupMember { Name = Final Fantasy II }: Name: Final Fantasy II }, { GroupMember { Name = Final Fantasy III }: Name: Final Fantasy III }, { GroupMember { Name = Final Fantasy IV }: Name: Final Fantasy IV }, { GroupMember { Name = Final Fantasy V }: Name: Final Fantasy V }, { GroupMember { Name = Final Fantasy VI }: Name: Final Fantasy VI }, { GroupMember { Name = Final Fantasy VII }: Name: Final Fantasy VII }, { GroupMember { Name = Final Fantasy VIII }: Name: Final Fantasy VIII }, { GroupMember { Name = Final Fantasy IX }: Name: Final Fantasy IX }, { GroupMember { Name = Final Fantasy X }: Name: Final Fantasy X }, { GroupMember { Name = Final Fantasy XI }: Name: Final Fantasy XI }, { GroupMember { Name = Final Fantasy XII }: Name: Final Fantasy XII }, { GroupMember { Name = Final Fantasy XIII }: Name: Final Fantasy XIII }, { GroupMember { Name = Final Fantasy XIV }: Name: Final Fantasy XIV }, { GroupMember { Name = Final Fantasy XV }: Name: Final Fantasy XV } ]"


# Reading events from a stream

The aggregate root was designed to be able to read from a stream and get back to current state, the output should be exactly what was rendered in the previous step.

In [ ]:
group = await Group.ReadAsync(stream);
display(group);

Id,Members
{ GroupId { Name = Final Fantasty Games }: Name: Final Fantasty Games },"[ { GroupMember { Name = Final Fantasy I }: Name: Final Fantasy I }, { GroupMember { Name = Final Fantasy II }: Name: Final Fantasy II }, { GroupMember { Name = Final Fantasy III }: Name: Final Fantasy III }, { GroupMember { Name = Final Fantasy IV }: Name: Final Fantasy IV }, { GroupMember { Name = Final Fantasy V }: Name: Final Fantasy V }, { GroupMember { Name = Final Fantasy VI }: Name: Final Fantasy VI }, { GroupMember { Name = Final Fantasy VII }: Name: Final Fantasy VII }, { GroupMember { Name = Final Fantasy VIII }: Name: Final Fantasy VIII }, { GroupMember { Name = Final Fantasy IX }: Name: Final Fantasy IX }, { GroupMember { Name = Final Fantasy X }: Name: Final Fantasy X }, { GroupMember { Name = Final Fantasy XI }: Name: Final Fantasy XI }, { GroupMember { Name = Final Fantasy XII }: Name: Final Fantasy XII }, { GroupMember { Name = Final Fantasy XIII }: Name: Final Fantasy XIII }, { GroupMember { Name = Final Fantasy XIV }: Name: Final Fantasy XIV }, { GroupMember { Name = Final Fantasy XV }: Name: Final Fantasy XV } ]"


# Generate some more events

Final Fantasy XIV was never really a game until A Realm Reborn was released.

In [ ]:
group.RemoveMember("Final Fantasy XIV");
group.AddMember("Final Fantasty XIV - A Realm Reborn");
await group.SaveAsync(stream);

display(group);

Id,Members
{ GroupId { Name = Final Fantasty Games }: Name: Final Fantasty Games },"[ { GroupMember { Name = Final Fantasy I }: Name: Final Fantasy I }, { GroupMember { Name = Final Fantasy II }: Name: Final Fantasy II }, { GroupMember { Name = Final Fantasy III }: Name: Final Fantasy III }, { GroupMember { Name = Final Fantasy IV }: Name: Final Fantasy IV }, { GroupMember { Name = Final Fantasy V }: Name: Final Fantasy V }, { GroupMember { Name = Final Fantasy VI }: Name: Final Fantasy VI }, { GroupMember { Name = Final Fantasy VII }: Name: Final Fantasy VII }, { GroupMember { Name = Final Fantasy VIII }: Name: Final Fantasy VIII }, { GroupMember { Name = Final Fantasy IX }: Name: Final Fantasy IX }, { GroupMember { Name = Final Fantasy X }: Name: Final Fantasy X }, { GroupMember { Name = Final Fantasy XI }: Name: Final Fantasy XI }, { GroupMember { Name = Final Fantasy XII }: Name: Final Fantasy XII }, { GroupMember { Name = Final Fantasy XIII }: Name: Final Fantasy XIII }, { GroupMember { Name = Final Fantasy XV }: Name: Final Fantasy XV }, { GroupMember { Name = Final Fantasty XIV - A Realm Reborn }: Name: Final Fantasty XIV - A Realm Reborn } ]"


# Show the events from a stream

You can see each version, the type of event, and the event data.

In [ ]:
var events = await stream.ReadAsync().ToListAsync();
display(events);

index,Version,Timestamp,Type,Data
0,0,2021-10-11 19:21:30Z,Submission#7+GroupEvent+Created,"{""Id"":{""Name"":""Final Fantasty Games""}}"
1,1,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy I""}}"
2,2,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy II""}}"
3,3,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy III""}}"
4,4,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy IV""}}"
5,5,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy V""}}"
6,6,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy VI""}}"
7,7,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy VII""}}"
8,8,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy VIII""}}"
9,9,2021-10-11 19:21:30Z,Submission#7+GroupEvent+AddedMember,"{""Id"":{""Name"":""Final Fantasty Games""},""Member"":{""Name"":""Final Fantasy IX""}}"


# Snapshots

This is a version of the `Group` aggregate root that uses snapshots to reduce the load on how many events need to be read to rehydrate the state. Notice the changes in `ReadAsync` and `SaveAsync` and the addition of the GroupState object.

In [ ]:
public record GroupState(GroupId Id, IEnumerable<GroupMember> Members);

public class Group
{
    public GroupId Id { get; private set; }
    public IEnumerable<GroupMember> Members => _members.AsEnumerable();
    
    private readonly List<GroupMember> _members = new();
    private readonly List<GroupEvent> _uncommittedEvents = new();

    private Group()
    {
    }

    private void AddEvent(GroupEvent e)
    {
        ApplyEvent(e);
        _uncommittedEvents.Add(e);
    }

    private void ApplyEvent(GroupEvent e)
    {
        switch (e)
        {
            case GroupEvent.Created created:
                Apply(created);
                break;
            case GroupEvent.AddedMember addedMember:
                Apply(addedMember);
                break;
            case GroupEvent.RemovedMember removedMember:
                Apply(removedMember);
                break;
             default:
                 throw new InvalidOperationException($"Unsupported event: {e.GetType().FullName}");
        }
    }

    public static Group Create(string name)
    {
        var group = new Group();
        group.AddEvent(new GroupEvent.Created(new GroupId(name)));
        return group;
    }

    private void Apply(GroupEvent.Created e)
        => Id = e.Id;

    public void AddMember(string name)
    {
        if (!_members.Any(x => x.Name == name))
        {
            var member = new GroupMember(name);
            var e = new GroupEvent.AddedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.AddedMember e)
        => _members.Add(e.Member);

    public void RemoveMember(string name)
    {
        var member = _members.FirstOrDefault(x => x.Name == name);

        if (member is not null)
        {
            var e = new GroupEvent.RemovedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.RemovedMember e)
        => _members.Remove(e.Member);

    public static async Task<Group?> ReadAsync(EventStream stream)
    {
        var group = new Group();
        var snapshot = await stream.ReadSnapshotAsync<GroupState>();
        var state = snapshot.State;
        
        // Hydrate the state from the snapshot if available
        if (state is not null)
        {
            group.Id = state.Id;
            group._members.AddRange(state.Members);
        }
        
        // Read only the events after the snapshot if the snapshot is available
        var events = state is null
            ? await stream.ReadAsync().ToListAsync()
            : await snapshot.ReadAsync().ToListAsync();

        if (!events.Any() && state is null)
        {
            return null;
        }

        foreach(var e in events)
        {
            if (!GroupEvent.Deserializer.TryDeserialize<GroupEvent>(e, out var groupEvent))
            {
                throw new InvalidOperationException($"The event {e.Type} is not supported.");
            }

            group.ApplyEvent(groupEvent);
        }

        return group;
    }

    public async Task SaveAsync(EventStream stream)
    {
        if (_uncommittedEvents.Count > 0)
        {
            var version = await stream.AppendAsync(_uncommittedEvents);
            _uncommittedEvents.Clear();
            await stream.WriteSnapshotAsync(new GroupState(Id, _members), version);
        }
    }
}

# Snapshot usage

Usage of the aggregate root looks exactly the same, except now every time events are saved, a snapshot is added to get back to current state instantly during a read.

In [ ]:
var favorites = Group.Create("Favorite Final Fantasy Games");

favorites.AddMember("Final Fantasy VI");
favorites.AddMember("Final Fantasy VII");
favorites.AddMember("Final Fantasy X");
favorites.AddMember("Final Fantasy XV");

stream = await store.GetStreamAsync(favorites.Id.Name);

await favorites.SaveAsync(stream);

favorites = await Group.ReadAsync(stream);

display(favorites);

Id,Members
{ GroupId { Name = Favorite Final Fantasy Games }: Name: Favorite Final Fantasy Games },"[ { GroupMember { Name = Final Fantasy VI }: Name: Final Fantasy VI }, { GroupMember { Name = Final Fantasy VII }: Name: Final Fantasy VII }, { GroupMember { Name = Final Fantasy X }: Name: Final Fantasy X }, { GroupMember { Name = Final Fantasy XV }: Name: Final Fantasy XV } ]"


# Show the snapshot

The snapshot keeps track of the version it was generated from so that it can read events created after that version.

In [ ]:
var snapshot = await stream.ReadSnapshotAsync<GroupState>();
display(snapshot);
display(snapshot.State);

Version,State
{ EventStreamVersion { Value = 5 }: Value: 5 },"{ GroupState { Id = GroupId { Name = Favorite Final Fantasy Games }, Members = System.Collections.Generic.List`1[Submission#7+GroupMember] }: Id: { GroupId { Name = Favorite Final Fantasy Games }: Name: Favorite Final Fantasy Games }, Members: [ { GroupMember { Name = Final Fantasy VI }: Name: Final Fantasy VI }, { GroupMember { Name = Final Fantasy VII }: Name: Final Fantasy VII }, { GroupMember { Name = Final Fantasy X }: Name: Final Fantasy X }, { GroupMember { Name = Final Fantasy XV }: Name: Final Fantasy XV } ] }"


Id,Members
{ GroupId { Name = Favorite Final Fantasy Games }: Name: Favorite Final Fantasy Games },"[ { GroupMember { Name = Final Fantasy VI }: Name: Final Fantasy VI }, { GroupMember { Name = Final Fantasy VII }: Name: Final Fantasy VII }, { GroupMember { Name = Final Fantasy X }: Name: Final Fantasy X }, { GroupMember { Name = Final Fantasy XV }: Name: Final Fantasy XV } ]"
